In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import os.path as osp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

In [2]:
def mkdirs(d):
    if not osp.exists(d):
        os.makedirs(d)

In [3]:
img_dir = osp.join("data_raw","img1")
dataset_dir = "dataset"
annotations_dir = osp.join(dataset_dir, 'annotations')

In [4]:
imgs = os.listdir(img_dir)
imgs.sort()

annotations = os.listdir(annotations_dir)
annotations.sort()

In [5]:
len(imgs),len(annotations)

(429, 429)

In [6]:
def xywh2xyxy(x,y,w,h):
    x0 = x-w//2
    y0 = y-h//2
    x1 = x+w//2
    y1 = y+h//2
    return x0,y0,x1,y1

In [7]:
df = pd.DataFrame(columns=['class','img_name'])

In [8]:
def export_detection(annotation,img_name, image_dir,export_dir):
    global df
    img = cv2.imread(os.path.join(img_dir,img_name))
    tid = int(annotation['id'])
    final_dir = osp.join(export_dir,'imgs',str(tid))
    
    img_w = 1920
    img_h = 1080

    x = int(annotation['x']*(img_w))
    y = int(annotation['y']*(img_h))
    w = int(annotation['w']*(img_w))
    h = int(annotation['h']*(img_h))

    x0,y0,x1,y1 = xywh2xyxy(x,y,w,h)
    crop = img[y0:y1,x0:x1] 


    mkdirs(final_dir)
    cv2.imwrite(os.path.join(final_dir,img_name),crop)
    df = df.append({'class':tid, 'img_name':img_name},ignore_index=True)

    del img, crop


In [9]:
for i, a in zip(imgs,annotations):
    a_csv = pd.read_csv(osp.join(annotations_dir,a),names=['c','id','x','y','w','h'],delimiter=" ")
    a_csv.apply(lambda x: export_detection(x,i,img_dir,dataset_dir), axis=1)

In [10]:
df.to_csv(osp.join(dataset_dir, 'data.csv'),index=False,header=None)